In [1]:
from src.opensource.llama_frontend import chat_pipeline, get_clues
import pandas as pd
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferWindowMemory
from src.opensource.templates import SYS_LLAMA_TEMPLATE, INST_LLAMA_TEMPLATE, SYS_BLOOMZ_TEMPLATE, INST_BLOOMZ_TEMPLATE

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [2]:
conversation_buffer = ConversationBufferWindowMemory(k=2, memory_key="chat_history")

In [3]:
inst_template = PromptTemplate.from_template(INST_LLAMA_TEMPLATE)
inst_template = inst_template.format(state='Jharkhand')
print(inst_template)


    [INST] Name the object on the basis of the above clues from Jharkhand. After your guess, I will let you know if you are correct or not. If you're correct, do not make any further guesses and end the conversation. If you are wrong, make your second and final guess. I do not need to know your reasoning behind the answer. Just tell me the answer and nothing else. If you do not know the answer, say that you do not know the answer. Format your answer in the form of ANSWER: your_answer_here.[/INST]



In [4]:
df_jharkhand = pd.read_csv('/home/t-sahuja/cultural_artifacts/clues/jharkhand/jharkhand_clues_1.csv')

In [5]:
df_jharkhand

,artifact,clues
0,Birsa Munda National Park,It is a National Park.\nIt is located in Orman...
1,Jonha Falls,Waterfall located in <> in Ranchi district\nIt...
2,Hunduru Falls,Highest fall of Jharkhand.\nIt is a common wee...
3,Dhol,It is a musical intrument played with both han...
4,Tribal woodwork,It is a artform done on wood by the Tribes of ...
5,Netrahat,It is a hill station for weekend picinic spot\...
6,Juliee Park,It is very famous park in Jameshdpur\nIt is a ...
7,Baba Baidyanath Temple,It is a very famous Lord Shiva temple at Devga...
8,Betla National Park,"National Park and it is famous for elephant, t..."
9,PAITKAR PAINTINGS,A famous painting from a parrticular community...


In [6]:
df_eval = pd.DataFrame(columns=['guess1', 'guess2', 'ground_truth', 'clues'])

In [7]:
def get_outputs(df, df_eval):
    for i, row in df.iterrows():
        print(f"artifact--{i}---")
        clues = row['clues'].split('\n')
        artifact = row['artifact'].lower().strip()
        output = ""
        for j, clue in enumerate(clues):
            output += f"CLUE-{j+1}: {clue}\n"
            fin_clues = output.strip()
        
        print(fin_clues, "clues")
        agent = chat_pipeline(clue_list = fin_clues, prompt_text=SYS_LLAMA_TEMPLATE, conversation_buffer=conversation_buffer)
        guess1 = agent.predict(human_input=inst_template)
        guess1 = guess1.split(':')[1].strip().lower()
        print(guess1, "Guess1")
        if artifact in guess1:
            df_eval = df_eval.append({'guess1': guess1, 'guess2': 'NA', 'ground_truth': artifact, 'clues': row['clues']}, ignore_index=True)
            conversation_buffer.clear()
            continue
        else:
            guess2 = agent.predict(human_input="Your first guess is not correct")
            guess2 = guess2.split(':')[1].strip().lower()
            print(guess2, "Guess2")
            df_eval = df_eval.append({'guess1': guess1, 'guess2': guess2, 'ground_truth': artifact, 'clues': row['clues']}, ignore_index=True)
            conversation_buffer.clear()
    return df_eval
            

In [8]:
conversation_buffer.clear()

In [9]:
df_eval = get_outputs(df_jharkhand, df_eval)

artifact--0---
CLUE-1: It is a National Park.
CLUE-2: It is located in Ormanjhi
CLUE-3: It is named after a famous freedom fighter.
CLUE-4: It was a tribute to that freedom fighter
CLUE-5: clues
birsa munda national park Guess1
artifact--1---
CLUE-1: Waterfall located in <> in Ranchi district
CLUE-2: It is surrounded by both forest and hills.
CLUE-3: There is a Buddha temple there. clues
the social artifact i am guessing is the "ranchi hill" or "ranchi plateau". Guess1
my second and final guess is the "hundru falls". Guess2
artifact--2---
CLUE-1: Highest fall of Jharkhand.
CLUE-2: It is a common weekend picnic and tourist spot. clues
ranchi hill. Guess1
hazaribagh national park Guess2
artifact--3---
CLUE-1: It is a musical intrument played with both hands.
CLUE-2: It is integral to the tribal dance and hold importance for the tribes of Jharkhand
CLUE-3: clues


KeyboardInterrupt: 

In [ ]:
df_eval.to_csv('tamil_evals.csv', index=False)

In [ ]:
df_eval